## 0. Imports

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import logging
from google.cloud.storage import Client
from sklearn.model_selection import train_test_split
import datetime
import gcsfs
import pickle
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

## 1. Config work

#### EXECUTE THE FOLLOWING COMMAND ONLY ONCE

In [ ]:
MODEL_TIME_VERSION  = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [ ]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT = PROJECT_ID[0]
BUCKET_NAME = f"{PROJECT}-machine-learning"
BUCKET= f"gs://{PROJECT}-machine-learning"
RAW_DATA_FOLDER_NAME = "raw-data"
RAW_DATA_FOLDER_PATH = f"gs://{PROJECT}-machine-learning/raw-data"
ROOT='level-0-models'
MODEL_DIR=os.path.join(ROOT,'models').replace("\\","/")
PACKAGES_DIR=os.path.join(ROOT,'packages').replace("\\","/")
REGION = 'europe-west1'
MODEL_NAME = 'tweet_sentiment_classifier'

if not os.path.exists('./model-'+ MODEL_TIME_VERSION +'/'):
    os.makedirs('./model-'+ MODEL_TIME_VERSION +'/')
temp_model = './model-'+ MODEL_TIME_VERSION +'/'

In [ ]:
!gcloud config set project {PROJECT}

In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

## 2. Get data

### 2.1. Input data

Use a !gcloud storage command to create a bucket gs://{BUCKET_NAME}

In [ ]:
!gcloud storage "TO_FILL" gs://{BUCKET_NAME}

Download the data from https://we.tl/t-g8SNeXnPh5 (cf description in https://www.kaggle.com/kazanova/sentiment140).
You should get 4 files:
- glove.twitter.27B.25d.txt
- training_data.csv
- dataset_VA.csv
- dataset_VB.csv
Go to Cloud Storage, bucket, and use the "importer des fichiers" or "importer un dossier" to upload those files into the bucket gs://{BUCKET_NAME}/raw-data.

In [ ]:
sentiment_mapping={
    0:"negative",
    2:"neutral",
    4:"positive"
}

df_twitter = pd.read_csv("gs://"+BUCKET_NAME+"/raw-data/training_VA.csv",encoding="latin1", header=None)\
             .rename(columns={
                 0:"sentiment",
                 1:"id",
                 2:"time",
                 3:"query",
                 4:"username",
                 5:"text"
             })[["sentiment","text"]]

df_twitter["sentiment_label"] = df_twitter["sentiment"].map(sentiment_mapping)
df_twitter.head()

Plot a bar plot to visualize the distribution of sentiment_labels. Is the dataset balanced?

### 2.2. Data processing fn

In [ ]:
%%writefile preprocess.py

from tensorflow.python.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import text
import re

class TextPreprocessor(object):
    def _clean_line(self, text):
        text = re.sub(r"http\S+", "", text)
        text = re.sub(r"@[A-Za-z0-9]+", "", text)
        text = re.sub(r"#[A-Za-z0-9]+", "", text)
        text = text.replace("RT","")
        text = text.lower()
        text = text.strip()
        return text
    
    def __init__(self, vocab_size, max_sequence_length):
        self._vocab_size = vocab_size
        self._max_sequence_length = max_sequence_length
        self._tokenizer = None

    def fit(self, text_list):        
        # Create vocabulary from input corpus.
        text_list_cleaned = [self._clean_line(txt) for txt in text_list]
        tokenizer = text.Tokenizer(num_words=self._vocab_size)
        tokenizer.fit_on_texts(text_list)
        self._tokenizer = tokenizer

    def transform(self, text_list):        
        # Transform text to sequence of integers
        text_list = [self._clean_line(txt) for txt in text_list]
        text_sequence = self._tokenizer.texts_to_sequences(text_list)

        # Fix sequence length to max value. Sequences shorter than the length are
        # padded in the beginning and sequences longer are truncated
        # at the beginning.
        padded_text_sequence = sequence.pad_sequences(
          text_sequence, maxlen=self._max_sequence_length)
        return padded_text_sequence

Some small test:

In [ ]:
from preprocess import TextPreprocessor
processor = TextPreprocessor(5, 5)
processor.fit(['hello machine learning','test'])
processor.transform(['hello machine learning',"lol"])

### 2.3. Prep data

We wish to limit the size of the vocabulary used for tokenization to 25000.
The sentences with be truncated to a length of 50 words if they are too long, and passed to 50 if they are too short. 

In [ ]:
CLASSES = {'negative':0, 'positive': 1}  # label-to-int mapping
VOCAB_SIZE = "TO FILL WITH AN INTEGER"
MAX_SEQUENCE_LENGTH = "TO FILL WITH AN INTEGER"

In [ ]:
sents = df_twitter.text
labels = np.array(df_twitter.sentiment_label.map(CLASSES))

# Train and test split
X, X_test, y, y_test = train_test_split(sents, labels, test_size=0.2)
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.1)

# Create vocabulary from training corpus.
processor = TextPreprocessor(VOCAB_SIZE, MAX_SEQUENCE_LENGTH)
processor.fit(X_train)

# Preprocess the data
train_texts_vectorized = processor.transform(X_train)
eval_texts_vectorized = processor.transform(X_test)
validation_texts_vectorized = processor.transform(X_validation)

with open('./model-'+ MODEL_TIME_VERSION +'/processor_state.pkl', 'wb') as f:
    pickle.dump(processor, f)

## 3. Model

In [ ]:
# model parameters
LEARNING_RATE=.001
EMBEDDING_DIM=50
FILTERS=64
DROPOUT_RATE=0.5
POOL_SIZE=3
NUM_EPOCH=2
BATCH_SIZE=128
KERNEL_SIZES=[2,5,8]

### 3.1. Basic model

fill in the missing variable

In [ ]:
def create_model(vocab_size, embedding_dim, filters, kernel_sizes, dropout_rate, pool_size, embedding_matrix):
    
    # Input layer
    model_input = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

    # Embedding layer
    z = tf.keras.layers.Embedding(
        input_dim=vocab_size+1,
        output_dim="TO FILL",
        input_length=MAX_SEQUENCE_LENGTH,
        weights=[embedding_matrix]
    )(model_input)

    z = tf.keras.layers.Dropout(dropout_rate)(z)

    # Convolutional block
    conv_blocks = []
    for kernel_size in kernel_sizes:
        conv = tf.keras.layers.Convolution1D(
            filters=filters,
            kernel_size=kernel_size,
            padding="valid",
            activation="relu",
            bias_initializer='random_uniform',
            strides=1)(z)
        conv = tf.keras.layers.MaxPooling1D(pool_size=2)(conv)
        conv = tf.keras.layers.Flatten()(conv)
        conv_blocks.append(conv)
        
    z = tf.keras.layers.Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

    z = tf.keras.layers.Dropout(dropout_rate)(z)
    z = tf.keras.layers.Dense(100, activation="relu")(z)
    model_output = tf.keras.layers.Dense(1, activation="sigmoid")(z)

    model = tf.keras.models.Model(model_input, model_output)
    
    return model

### 3.2. Pretrained Glove embeddings

Embedding can be downloaded here: https://nlp.stanford.edu/projects/glove/, but you should already have the globe.twitter.27B.25d.txt file in your cloud storage.

Fill in the missing part to download the glove.twitter.27B.25d.txt from your cloud storage.

In [ ]:
client = Client()
bucket = "TO FILL"
blob = "TO FILL"
downloaded_file = blob.download_to_filename('raw-data/glove.twitter.27B.25d.txt')

In [ ]:
def get_coaefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coaefs(*o.strip().split()) for o in
                                                open("raw-data/glove.twitter.27B.25d.txt","r",encoding="utf8"))

In [ ]:
word_index = processor._tokenizer.word_index
nb_words = min(VOCAB_SIZE, len(word_index))
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= VOCAB_SIZE: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

You've got an error? Fix it!!

### 3.3. Create - compile - train

Fill in the missing arguments

In [ ]:
model = create_model(VOCAB_SIZE, EMBEDDING_DIM)

In [ ]:
# Compile model with learning parameters.
optimizer = tf.keras.optimizers.Nadam(lr=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])

In [ ]:
#keras train
history = model.fit(
    train_texts_vectorized, 
    y_train, 
    epochs=NUM_EPOCH, 
    batch_size=BATCH_SIZE,
    validation_data=(validation_texts_vectorized, y_validation),
    verbose=2,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_acc',
            min_delta=0.005,
            patience=3,
            factor=0.5),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0.005, 
            patience=5, 
            verbose=0, 
            mode='auto'
        ),
        tf.keras.callbacks.History()
    ]
)

In [ ]:
# test model : acc loss
[loss, acc] = model.evaluate(eval_texts_vectorized, y_test )

In [ ]:
scores = model.predict(eval_texts_vectorized)
predictions = np.array([int(np.round(i)) for i in scores ])
confusion_matrix=tf.math.confusion_matrix(predictions, y_test)

In [ ]:
# confusion matrix
print (str("matrix-co : "+str(confusion_matrix)))

In [ ]:
# std prediction 
np.std(scores)

In [ ]:
with open("history.pkl",'wb') as file:
    pickle.dump(history.history,file)

In [ ]:
tf.keras.models.save_model(model,temp_model)

Fill in the missing command to copy the temp_model directory to gcp storage

In [ ]:
!"TO FILL" -r {temp_model} {BUCKET}/{MODEL_DIR}/

## 4. Deployment

### 4.1. Prepare custom model prediction

In [ ]:
%%writefile model_prediction.py
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.python.lib.io import file_io

class CustomModelPrediction(object):

    def __init__(self, model, processor):
        # Class gets instantiated with a trained model file and a persisted processor
        self._model = model
        self._processor = processor

    def _postprocess(self, predictions):
    # Create an output signature
        labels = ['negative', 'positive']
        return [
            {
            "label":labels[int(np.round(prediction))],
            "score":float(np.round(prediction,4))
            } for prediction in predictions]

    def predict(self, instances, **kwargs):
    # Clean the data, make predictions and postprocess
        preprocessed_data = self._processor.transform(instances)
        predictions =  self._model.predict(preprocessed_data)
        labels = self._postprocess(predictions)
        return labels

    @classmethod
    def from_path(cls, model_dir):
    # Load the keras model and the persisted processor
        
        print ('test model')
        model = tf.keras.models.load_model(model_dir,custom_objects={'tf': tf})
    
    # I know, pickle is bad and I should feel bad
    
        with file_io.FileIO(os.path.join(model_dir, 'processor_state.pkl'), 'rb') as f:
            processor = pickle.load(f)

        return cls(model, processor)

Test

In [ ]:
requests = (["God I hate the north","god I love this"])

In [ ]:
BUCKET+'/'+MODEL_DIR+temp_model[1:]

In [ ]:
from model_prediction import CustomModelPrediction

classifier = CustomModelPrediction.from_path(BUCKET+'/'+MODEL_DIR+temp_model[1:])
results = classifier.predict(requests)
results

### 4.2. Package it

In [ ]:
MODEL_TIME_VERSION
print(MODEL_TIME_VERSION)

#### update VERSION in the cell below with the MODEL_TIME_VERSION above

In [ ]:
%%writefile setup.py

from setuptools import setup

MODEL_NAME = "tweet_sentiment_classifier"
REQUIRED_PACKAGES = ['gcsfs']
VERSION = '2022-06-xx-xx-xx-xx'

setup(
    name=MODEL_NAME,
    packages=[],
    include_package_data=False,
    version=VERSION,
    scripts=["preprocess.py", "model_prediction.py"]
)

Wrap it up and copy to GCP

Fill in the missing command

In [ ]:
!python setup.py sdist --formats=gztar
!"TO FILL" ./dist/{MODEL_NAME}-{MODEL_TIME_VERSION}.tar.gz {BUCKET}/{PACKAGES_DIR}/{MODEL_NAME}-{MODEL_TIME_VERSION}.tar.gz

You've got an error? Try to fix it!!!

## 5. Create model and version

In [ ]:
VERSION_NAME='V_' + MODEL_TIME_VERSION
RUNTIME_VERSION='2.5' # tensorflow version
MODEL_REGION='europe-west1'
id_model = 'model-'+MODEL_TIME_VERSION

list your existing ai-platform model using a gcloud ai-platform command

In [ ]:
!gcloud "TO FILL" --region global

#### If no model has been created before, fill the next gcloud ai-platform command to create a model, then run this command.

In [ ]:
!gcloud ai-platform "TO FILL" {MODEL_NAME} --regions {MODEL_REGION}

Fill the next command to create a new version of the model using beta ai-platform

In [ ]:
!gcloud "TO FILL" {VERSION_NAME} \
--model {MODEL_NAME} \
--origin {BUCKET}/{MODEL_DIR}/{id_model} \
--python-version 3.7 \
--runtime-version {RUNTIME_VERSION} \
--package-uris {BUCKET}/{PACKAGES_DIR}/{MODEL_NAME}-{MODEL_TIME_VERSION}.tar.gz \
--prediction-class=model_prediction.CustomModelPrediction \
--region global 

Oops, the fix you made just above might be useful again...

## 6. Testing

In [ ]:
requests = [
    "god this episode sucks",
    "meh, I kinda like it",
    "what were the writer thinking, omg it doesn't make any sense!",
    "omg! what a twist, who would've though :o!",
    "woohoow, sansa for the win!"
]

# JSON format the requests
request_data = {'instances': requests}

# Authenticate and call CMLE prediction API 
credentials = GoogleCredentials.get_application_default()

In [ ]:
%%time

api = discovery.build('ml', 'v1')
model_url = 'projects/{}/models/{}'.format(PROJECT, MODEL_NAME)
response = api.projects().predict(body=request_data, name=model_url).execute()
response["predictions"]